<a href="https://colab.research.google.com/github/achillesposiedon/TaxiGameRL/blob/master/TaxiGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-rl
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import gym # openAi gym
from gym import envs
import numpy as np 
import datetime
import keras 

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import sleep

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.


In [3]:
env = gym.make('Taxi-v3')
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
tf.__version__

'1.15.2'

In [5]:
#random action evaluation
rdrwrd=0
obs=env.reset()
env.render()

for i in range(9):
  action=env.action_space.sample()# pick random action from sample space
  obs,rewrd,done,info=env.step(action)
  rdrwrd+=rewrd
  env.render()

print(rdrwrd)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
-27


In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(6)
Discrete(500)


In [7]:
env.env.s=42
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


In [8]:
#value iteration
naction=env.action_space.n
nstates=env.observation_space.n

stateval=np.zeros([nstates])
optimalpolicy=np.zeros([nstates], dtype=int)

gamma=0.9
sig_improv=0.0001

def optaction(state):
  opact=0
  best_value = float('-inf')

  for i in range(0,naction):
    env.env.s=state
    state_new, rew_new, done, info = env.step(i) #take the action
    val=rew_new+gamma*stateval[state_new]
    
    if val>best_value:
      best_value=val
      optact=i

  return optact
iteration=0
while True:
  delta=0
  for s in range(0,nstates):
    valold=stateval[s]
    action=optaction(s)

    env.env.s=s #set the state s0
    statenew,rewardnew,done,info=env.step(action)

    stateval[s]=rewardnew+gamma*stateval[statenew]
    optimalpolicy[s]=action

    delta=max(delta, np.abs(valold-stateval[s]))
  
  iteration+=1

  if delta<sig_improv:
    print(iteration)
    break

63


In [9]:
reward=0
obs=env.reset()
env.render()
done=False

while done!= True:
  action=optimalpolicy[obs]
  obs,rew,done,info =env.step(action)
  reward+=rew
  env.render()

print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
12


In [20]:
#Qlearning algo
env = gym.make('Taxi-v3')
nactions=env.action_space.n
nstates=env.observation_space.n

qtable=np.zeros([nstates,nactions])

gamma=0.9 #discount factor
alpha=0.9 #learning rate

for episode in range(1,1001):
  done=False
  reward=0
  obs=env.reset()
  while done!=True:
    action=np.argmax(qtable[obs])
    newobs,newrew,done,info=env.step(action)
    qtable[obs,action]+=alpha*(newrew+gamma*np.max(qtable[newobs])-qtable[obs,action])

    reward+=newrew
    obs=newobs

  if episode % 50 == 0:
    print(episode,reward)

50 3
100 -67
150 5
200 1
250 8
300 -31
350 7
400 9
450 6
500 9
550 5
600 7
650 8
700 7
750 5
800 9
850 8
900 12
950 3
1000 9


In [22]:
rew=0
obs=env.reset()
env.render()
done=False

while done!= True:
  action=np.argmax(qtable[obs])
  obs,newrwrd,done,info=env.step(action)
  rew+=newrwrd
  env.render()

print(rew)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |

In [23]:
env = gym.make('FrozenLake-v0')
rew_tot=0
obs= env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [39]:
env = gym.make('FrozenLake-v0')
env.reset()

nactions = env.action_space.n
nstates = env.observation_space.n
qtable = np.zeros([nstates, nactions]) #You could also make this dynamic if you don't know all games states upfront

gamma = 0.95 # discount factor
alpha = 0.9 # learning rate
epsilon = 0.1 #

for episode in range(1,500000001):
    done = False
    obs = env.reset()

    while done != True:
        
        if np.random.rand(1) < epsilon:
            # explore
            action = env.action_space.sample()
        
        else:
            # exploit
            action = np.argmax(qtable[obs])
        newobs, newrew, done, info = env.step(action) #take the action
        qtable[obs,action] += alpha * (newrew + gamma * np.max(qtable[newobs]) - qtable[obs,action]) #Update Q-marix using Bellman equation
        obs = newobs   
        
    if episode % 5000 == 0:
        #report every 5000 steps, test 100 games to get avarage point score for statistics and verify if it is solve
        rew_average = 0
        for i in range(1000):
            obs= env.reset()
            done=False
            while done != True: 
                action = np.argmax(qtable[obs])
                obs, rew, done, info = env.step(action) #take step using selected action
                rew_average += rew
        rew_average=rew_average/100
        print('Episode {} avarage reward: {}'.format(episode,rew_average))
        
        if rew_average > 0.8:
            # FrozenLake-v0 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.
            # Test it on 0.8 so it is not a one-off lucky shot solving it
            print("Frozen lake solved")
            break
 

Episode 5000 avarage reward: 0.0
Episode 10000 avarage reward: 0.0
Episode 15000 avarage reward: 0.0
Episode 20000 avarage reward: 0.0
Episode 25000 avarage reward: 0.0
Episode 30000 avarage reward: 0.0
Episode 35000 avarage reward: 7.46
Frozen lake solved


In [40]:
rew_tot=0.
obs= env.reset()
done=False
while done != True: 
    action = np.argmax(qtable[obs])
    obs, rew, done, info = env.step(action) #take step using selected action
    rew_tot += rew
    env.render()

print("Reward:", rew_tot) 

  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 1.0


In [41]:
qtable

array([[1.84138084e-01, 1.34418052e-01, 1.40227212e-01, 4.62075522e-02],
       [2.95439159e-02, 7.06825015e-09, 1.39560153e-02, 1.34090167e-01],
       [2.24767111e-03, 2.53548044e-03, 2.82370603e-03, 5.16325569e-03],
       [4.21648118e-04, 2.83552223e-05, 3.30042452e-04, 2.95483033e-03],
       [3.05821899e-01, 4.61126312e-02, 5.66821970e-03, 3.50200650e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.83729427e-02, 4.59670632e-05, 5.24064700e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.80035862e-04, 7.00575330e-02, 4.23479670e-03, 1.64570842e-01],
       [1.47247846e-01, 5.77331353e-01, 8.06307437e-02, 1.08569955e-03],
       [1.48542130e-01, 6.74398284e-03, 7.27385914e-03, 3.00296079e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.23809297e-03, 6.37243396e-02, 9.10370527e